In [1]:
import time
import logging
import os
from tqdm import tqdm
import numpy as np
import pandas as pd

from tinkoff.invest import Client, SecurityTradingStatus, CandleInterval
from tinkoff.invest.services import InstrumentsService
from tinkoff.invest.utils import quotation_to_decimal, now

from datetime import timedelta
import datetime

import pickle
def dump_pkl(data, filename):
  with open(filename, 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
def load_pkl(filename):
  with open(filename, 'rb') as handle:
    data = pickle.load(handle)
  return data

from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

import requests

# 1. Global params

In [3]:
TOKEN = 'xxx'
with Client(TOKEN) as client:
    print(client.users.get_accounts())

GetAccountsResponse(accounts=[Account(id='2169433725', type=<AccountType.ACCOUNT_TYPE_TINKOFF: 1>, name='M13', status=<AccountStatus.ACCOUNT_STATUS_OPEN: 2>, opened_date=datetime.datetime(2022, 4, 27, 0, 0, tzinfo=datetime.timezone.utc), closed_date=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), access_level=<AccessLevel.ACCOUNT_ACCESS_LEVEL_READ_ONLY: 2>)])


In [4]:
BOT_TOKEN = '7598372117:AAHKSc-jHo02zSzJ-NOoZ8GloOqdbNySeZw'
BOT_CHAT_ID = '131510115'

In [5]:
MODEL_FEATURES1 = ['close/close_w5_ma',
 'close/close_w10_ma',
 'close/close_w20_ma',
 'close/close_w5_expma',
 'close/close_w10_expma',
 'close_w5_min/close']


MODEL_FEATURES2 = ['close/close_w10_ma',
 'close/close_w30_ma',
 'close/close_w10_expma',
 'close/close_w30_expma',
 'close_w10_max/close',
 'close/close_1hour_w10_expma']

MODEL_FEATURES3 = ['close/close_w10_ma',
 'close/close_w20_ma',
 'close/close_w30_ma',
 'close/close_w10_expma',
 'close/close_w20_expma',
 'close/close_w30_expma']

FEATURES = list(set(MODEL_FEATURES1 + MODEL_FEATURES2 + MODEL_FEATURES3))

In [6]:
len(FEATURES)

11

In [7]:

COUNT_POINTS = 120




STOCKS = ['AFLT',
 'IRAO',
 'MVID',
 'SPBE',
 'MTLRP',
 'SFIN',
 'FEES',
 'LKOH',
 'FESH',
 'MDMG',
 'VKCO',
 'UPRO',
 'SELG',
 'SGZH',
 'ENPG',
 'LSRG',
 'X5',
 'NMTP',
 'PLZL',
 'TATN',
 'AFKS',
 'GMKN',
 'RNFT',
 'YDEX',
 'UWGN',
 'TGKN',
 'HYDR',
 'ABIO',
 'WUSH',
 'GTRK',
 'NLMK',
 'TATNP',
 'ALRS',
 'VSMO',
 'TRNFP',
 'MGNT',
 'IRKT',
 'MTLR',
 'HEAD',
 'RUAL',
 'SIBN',
 'SNGS',
 'NVTK',
 'RASP',
 'VTBR',
 'PHOR',
 'T',
 'TRMK',
 'MOEX',
 'SBERP',
 'CHMF',
 'RTKM',
 'SMLT',
 'UNAC',
 'GAZP',
 'ROSN',
 'BELU',
 'FLOT',
 'PIKK',
 'SVAV',
 'POSI',
 'SNGSP',
 'MTSS',
 'MAGN',
 'SBER',
 'BANEP',
 'BSPB']

In [8]:
len(STOCKS)

67

In [9]:
GROUPS_15MIN = [
        #windows: 
        #close: w1, w5, w10, w20, w30, w60, w120
        #volume: w1, w5, w10, w20
        #tmos_close: w1, w5, w10, w20, w30, w60, w120
         ['close_w1_roc', 'close_w5_alpha', 'close_w10_alpha', 'close_w20_alpha', 'close_w30_alpha', 'close_w60_alpha', 'close_w120_alpha'],
         ['volume_w1_roc', 'volume_w5_alpha', 'volume_w10_alpha', 'volume_w20_alpha'],
         ['tmos_close_w1_roc', 'tmos_close_w5_alpha', 'tmos_close_w10_alpha', 'tmos_close_w20_alpha', 'tmos_close_w30_alpha', 'tmos_close_w60_alpha', 'tmos_close_w120_alpha'],
    
    
         ['close_w1_roc', 'close_w5_roc', 'close_w10_roc', 'close_w20_roc', 'close_w30_roc', 'close_w60_roc', 'close_w120_roc'],
         ['volume_w1_roc', 'volume_w5_roc', 'volume_w10_roc', 'volume_w20_roc'],
         ['tmos_close_w1_roc', 'tmos_close_w5_roc', 'tmos_close_w10_roc', 'tmos_close_w20_roc', 'tmos_close_w30_roc', 'tmos_close_w60_roc', 'tmos_close_w120_roc'],

    
         ['close_w5_mean_abs_pct', 'close_w10_mean_abs_pct', 'close_w20_mean_abs_pct', 'close_w30_mean_abs_pct', 'close_w60_mean_abs_pct', 'close_w120_mean_abs_pct'],
         ['volume_w5_mean_abs_pct', 'volume_w10_mean_abs_pct', 'volume_w20_mean_abs_pct'],
         ['tmos_close_w5_mean_abs_pct', 'tmos_close_w10_mean_abs_pct', 'tmos_close_w20_mean_abs_pct', 'tmos_close_w30_mean_abs_pct', 'tmos_close_w60_mean_abs_pct', 'tmos_close_w120_mean_abs_pct'],


         ['close_w5_std', 'close_w10_std', 'close_w20_std', 'close_w30_std', 'close_w60_std', 'close_w120_std'],
         ['volume_w5_std', 'volume_w10_std', 'volume_w20_std'],
         ['tmos_close_w5_std', 'tmos_close_w10_std', 'tmos_close_w20_std', 'tmos_close_w30_std', 'tmos_close_w60_std', 'tmos_close_w120_std'],


         ['close_w5_norm_std', 'close_w10_norm_std', 'close_w20_norm_std', 'close_w30_norm_std', 'close_w60_norm_std', 'close_w120_norm_std'],
         ['volume_w5_norm_std', 'volume_w10_norm_std', 'volume_w20_norm_std'],
         ['tmos_close_w5_norm_std', 'tmos_close_w10_norm_std', 'tmos_close_w20_norm_std', 'tmos_close_w30_norm_std', 'tmos_close_w60_norm_std', 'tmos_close_w120_norm_std'],


         ['close_w5_rsi', 'close_w10_rsi', 'close_w20_rsi', 'close_w30_rsi', 'close_w60_rsi', 'close_w120_rsi'],
         ['volume_w5_rsi', 'volume_w10_rsi', 'volume_w20_rsi'],
         ['tmos_close_w5_rsi', 'tmos_close_w10_rsi', 'tmos_close_w20_rsi', 'tmos_close_w30_rsi', 'tmos_close_w60_rsi', 'tmos_close_w120_rsi'],


         ['close', 'close_w5_ma', 'close_w10_ma', 'close_w20_ma', 'close_w30_ma', 'close_w60_ma', 'close_w120_ma'],
         ['volume', 'volume_w5_ma', 'volume_w10_ma', 'volume_w20_ma'],
         ['tmos_close', 'tmos_close_w5_ma', 'tmos_close_w10_ma', 'tmos_close_w20_ma', 'tmos_close_w30_ma', 'tmos_close_w60_ma', 'tmos_close_w120_ma'],


         ['close', 'close_w5_expma', 'close_w10_expma', 'close_w20_expma', 'close_w30_expma', 'close_w60_expma', 'close_w120_expma'],
         ['volume', 'volume_w5_expma', 'volume_w10_expma', 'volume_w20_expma'],
         ['tmos_close', 'tmos_close_w5_expma', 'tmos_close_w10_expma', 'tmos_close_w20_expma', 'tmos_close_w30_expma', 'tmos_close_w60_expma', 'tmos_close_w120_expma'],


         {'close' : ['close_w5_min', 'close_w10_min', 'close_w20_min', 'close_w30_min', 'close_w60_min', 'close_w120_min']},
         {'volume' : ['volume_w5_min', 'volume_w10_min', 'volume_w20_min']},
         {'tmos_close' : ['tmos_close_w5_min', 'tmos_close_w10_min', 'tmos_close_w20_min', 'tmos_close_w30_min', 'tmos_close_w60_min', 'tmos_close_w120_min']},

    
         {'close' : ['close_w5_max', 'close_w10_max', 'close_w20_max', 'close_w30_max', 'close_w60_max', 'close_w120_max']},
         {'volume' : ['volume_w5_max', 'volume_w10_max', 'volume_w20_max']},
         {'tmos_close' : ['tmos_close_w5_max', 'tmos_close_w10_max', 'tmos_close_w20_max', 'tmos_close_w30_max', 'tmos_close_w60_max', 'tmos_close_w120_max']},



        #w5
         {'close' : [  'close_w5_ma_low_2std', 'close_w5_ma_up_2std', 'close_w5_ma_low_3std', 'close_w5_ma_up_3std']},
         {'volume' : [  'volume_w5_ma_low_2std', 'volume_w5_ma_up_2std', 'volume_w5_ma_low_3std', 'volume_w5_ma_up_3std']},
         {'tmos_close' : [  'tmos_close_w5_ma_low_2std', 'tmos_close_w5_ma_up_2std', 'tmos_close_w5_ma_low_3std', 'tmos_close_w5_ma_up_3std']},
        #w10
         {'close' : [  'close_w10_ma_low_2std', 'close_w10_ma_up_2std', 'close_w10_ma_low_3std', 'close_w10_ma_up_3std']},
         {'volume' : [  'volume_w10_ma_low_2std', 'volume_w10_ma_up_2std', 'volume_w10_ma_low_3std', 'volume_w10_ma_up_3std']},
         {'tmos_close' : [  'tmos_close_w10_ma_low_2std', 'tmos_close_w10_ma_up_2std', 'tmos_close_w10_ma_low_3std', 'tmos_close_w10_ma_up_3std']},
        #w20
         {'close' : [  'close_w20_ma_low_2std', 'close_w20_ma_up_2std', 'close_w20_ma_low_3std', 'close_w20_ma_up_3std']},
         {'volume' : [  'volume_w20_ma_low_2std', 'volume_w20_ma_up_2std', 'volume_w20_ma_low_3std', 'volume_w20_ma_up_3std']},
         {'tmos_close' : [  'tmos_close_w20_ma_low_2std', 'tmos_close_w20_ma_up_2std', 'tmos_close_w20_ma_low_3std', 'tmos_close_w20_ma_up_3std']},
        #w30
         {'close' : [  'close_w30_ma_low_2std', 'close_w30_ma_up_2std', 'close_w30_ma_low_3std', 'close_w30_ma_up_3std']},
         {'tmos_close' : [  'tmos_close_w30_ma_low_2std', 'tmos_close_w30_ma_up_2std', 'tmos_close_w30_ma_low_3std', 'tmos_close_w30_ma_up_3std']},
        #w60
         {'close' : [  'close_w60_ma_low_2std', 'close_w60_ma_up_2std', 'close_w60_ma_low_3std', 'close_w60_ma_up_3std']},
         {'tmos_close' : [  'tmos_close_w60_ma_low_2std', 'tmos_close_w60_ma_up_2std', 'tmos_close_w60_ma_low_3std', 'tmos_close_w60_ma_up_3std']},
        #w120
         {'close' : [  'close_w120_ma_low_2std', 'close_w120_ma_up_2std', 'close_w120_ma_low_3std', 'close_w120_ma_up_3std']},
         {'tmos_close' : [  'tmos_close_w120_ma_low_2std', 'tmos_close_w120_ma_up_2std', 'tmos_close_w120_ma_low_3std', 'tmos_close_w120_ma_up_3std']},


        #w5
        ['close_w5_min', 'close_w5_max'],
        ['volume_w5_min', 'volume_w5_max'],
        ['tmos_close_w5_min', 'tmos_close_w5_max'],
        #w10
        ['close_w10_min', 'close_w10_max'],
        ['volume_w10_min', 'volume_w10_max'],
        ['tmos_close_w10_min', 'tmos_close_w10_max'],
        #w20
        ['close_w20_min', 'close_w20_max'],
        ['volume_w20_min', 'volume_w20_max'],
        ['tmos_close_w20_min', 'tmos_close_w20_max'],
        #w30
        ['close_w30_min', 'close_w30_max'],
        ['tmos_close_w30_min', 'tmos_close_w30_max'],
        #w60
        ['close_w60_min', 'close_w60_max'],
        ['tmos_close_w60_min', 'tmos_close_w60_max'],
        #w120
        ['close_w120_min', 'close_w120_max'],
        ['tmos_close_w120_min', 'tmos_close_w120_max'],
]


GROUPS_1HOUR = [
        #windows: 
        #close: w1, w5, w10, w20, w30, w60, w120
        #volume: w1, w5, w10, w20
        #tmos_close: w1, w5, w10, w20, w30, w60, w120
         ['close_1hour_w1_roc', 'close_1hour_w5_alpha', 'close_1hour_w10_alpha', 'close_1hour_w20_alpha', 'close_1hour_w30_alpha', 'close_1hour_w60_alpha', 'close_1hour_w120_alpha'],
         ['volume_1hour_w1_roc', 'volume_1hour_w5_alpha', 'volume_1hour_w10_alpha', 'volume_1hour_w20_alpha'],
         ['tmos_close_1hour_w1_roc', 'tmos_close_1hour_w5_alpha', 'tmos_close_1hour_w10_alpha', 'tmos_close_1hour_w20_alpha', 'tmos_close_1hour_w30_alpha', 'tmos_close_1hour_w60_alpha', 'tmos_close_1hour_w120_alpha'],
    
         ['close_1hour_w1_roc', 'close_1hour_w5_roc', 'close_1hour_w10_roc', 'close_1hour_w20_roc', 'close_1hour_w30_roc', 'close_1hour_w60_roc', 'close_1hour_w120_roc'],
         ['volume_1hour_w1_roc', 'volume_1hour_w5_roc', 'volume_1hour_w10_roc', 'volume_1hour_w20_roc'],
         ['tmos_close_1hour_w1_roc', 'tmos_close_1hour_w5_roc', 'tmos_close_1hour_w10_roc', 'tmos_close_1hour_w20_roc', 'tmos_close_1hour_w30_roc', 'tmos_close_1hour_w60_roc', 'tmos_close_1hour_w120_roc'],

         ['close_1hour_w5_mean_abs_pct', 'close_1hour_w10_mean_abs_pct', 'close_1hour_w20_mean_abs_pct', 'close_1hour_w30_mean_abs_pct', 'close_1hour_w60_mean_abs_pct', 'close_1hour_w120_mean_abs_pct'],
         ['volume_1hour_w5_mean_abs_pct', 'volume_1hour_w10_mean_abs_pct', 'volume_1hour_w20_mean_abs_pct'],
         ['tmos_close_1hour_w5_mean_abs_pct', 'tmos_close_1hour_w10_mean_abs_pct', 'tmos_close_1hour_w20_mean_abs_pct', 'tmos_close_1hour_w30_mean_abs_pct', 'tmos_close_1hour_w60_mean_abs_pct', 'tmos_close_1hour_w120_mean_abs_pct'],


         ['close_1hour_w5_std', 'close_1hour_w10_std', 'close_1hour_w20_std', 'close_1hour_w30_std', 'close_1hour_w60_std', 'close_1hour_w120_std'],
         ['volume_1hour_w5_std', 'volume_1hour_w10_std', 'volume_1hour_w20_std'],
         ['tmos_close_1hour_w5_std', 'tmos_close_1hour_w10_std', 'tmos_close_1hour_w20_std', 'tmos_close_1hour_w30_std', 'tmos_close_1hour_w60_std', 'tmos_close_1hour_w120_std'],


         ['close_1hour_w5_norm_std', 'close_1hour_w10_norm_std', 'close_1hour_w20_norm_std', 'close_1hour_w30_norm_std', 'close_1hour_w60_norm_std', 'close_1hour_w120_norm_std'],
         ['volume_1hour_w5_norm_std', 'volume_1hour_w10_norm_std', 'volume_1hour_w20_norm_std'],
         ['tmos_close_1hour_w5_norm_std', 'tmos_close_1hour_w10_norm_std', 'tmos_close_1hour_w20_norm_std', 'tmos_close_1hour_w30_norm_std', 'tmos_close_1hour_w60_norm_std', 'tmos_close_1hour_w120_norm_std'],


         ['close_1hour_w5_rsi', 'close_1hour_w10_rsi', 'close_1hour_w20_rsi', 'close_1hour_w30_rsi', 'close_1hour_w60_rsi', 'close_1hour_w120_rsi'],
         ['volume_1hour_w5_rsi', 'volume_1hour_w10_rsi', 'volume_1hour_w20_rsi'],
         ['tmos_close_1hour_w5_rsi', 'tmos_close_1hour_w10_rsi', 'tmos_close_1hour_w20_rsi', 'tmos_close_1hour_w30_rsi', 'tmos_close_1hour_w60_rsi', 'tmos_close_1hour_w120_rsi'],


         ['close', 'close_1hour_w5_ma', 'close_1hour_w10_ma', 'close_1hour_w20_ma', 'close_1hour_w30_ma', 'close_1hour_w60_ma', 'close_1hour_w120_ma'],
         ['volume', 'volume_1hour_w5_ma', 'volume_1hour_w10_ma', 'volume_1hour_w20_ma'],
         ['tmos_close', 'tmos_close_1hour_w5_ma', 'tmos_close_1hour_w10_ma', 'tmos_close_1hour_w20_ma', 'tmos_close_1hour_w30_ma', 'tmos_close_1hour_w60_ma', 'tmos_close_1hour_w120_ma'],


         ['close', 'close_1hour_w5_expma', 'close_1hour_w10_expma', 'close_1hour_w20_expma', 'close_1hour_w30_expma', 'close_1hour_w60_expma', 'close_1hour_w120_expma'],
         ['volume', 'volume_1hour_w5_expma', 'volume_1hour_w10_expma', 'volume_1hour_w20_expma'],
         ['tmos_close', 'tmos_close_1hour_w5_expma', 'tmos_close_1hour_w10_expma', 'tmos_close_1hour_w20_expma', 'tmos_close_1hour_w30_expma', 'tmos_close_1hour_w60_expma', 'tmos_close_1hour_w120_expma'],


         {'close' : ['close_1hour_w5_min', 'close_1hour_w10_min', 'close_1hour_w20_min', 'close_1hour_w30_min', 'close_1hour_w60_min', 'close_1hour_w120_min']},
         {'volume' : ['volume_1hour_w5_min', 'volume_1hour_w10_min', 'volume_1hour_w20_min']},
         {'tmos_close' : ['tmos_close_1hour_w5_min', 'tmos_close_1hour_w10_min', 'tmos_close_1hour_w20_min', 'tmos_close_1hour_w30_min', 'tmos_close_1hour_w60_min', 'tmos_close_1hour_w120_min']},

    
         {'close' : ['close_1hour_w5_max', 'close_1hour_w10_max', 'close_1hour_w20_max', 'close_1hour_w30_max', 'close_1hour_w60_max', 'close_1hour_w120_max']},
         {'volume' : ['volume_1hour_w5_max', 'volume_1hour_w10_max', 'volume_1hour_w20_max']},
         {'tmos_close' : ['tmos_close_1hour_w5_max', 'tmos_close_1hour_w10_max', 'tmos_close_1hour_w20_max', 'tmos_close_1hour_w30_max', 'tmos_close_1hour_w60_max', 'tmos_close_1hour_w120_max']},



        #w5
         {'close' : [  'close_1hour_w5_ma_low_2std', 'close_1hour_w5_ma_up_2std', 'close_1hour_w5_ma_low_3std', 'close_1hour_w5_ma_up_3std']},
         {'volume' : [  'volume_1hour_w5_ma_low_2std', 'volume_1hour_w5_ma_up_2std', 'volume_1hour_w5_ma_low_3std', 'volume_1hour_w5_ma_up_3std']},
         {'tmos_close' : [  'tmos_close_1hour_w5_ma_low_2std', 'tmos_close_1hour_w5_ma_up_2std', 'tmos_close_1hour_w5_ma_low_3std', 'tmos_close_1hour_w5_ma_up_3std']},
        #w10
         {'close' : [  'close_1hour_w10_ma_low_2std', 'close_1hour_w10_ma_up_2std', 'close_1hour_w10_ma_low_3std', 'close_1hour_w10_ma_up_3std']},
         {'volume' : [  'volume_1hour_w10_ma_low_2std', 'volume_1hour_w10_ma_up_2std', 'volume_1hour_w10_ma_low_3std', 'volume_1hour_w10_ma_up_3std']},
         {'tmos_close' : [  'tmos_close_1hour_w10_ma_low_2std', 'tmos_close_1hour_w10_ma_up_2std', 'tmos_close_1hour_w10_ma_low_3std', 'tmos_close_1hour_w10_ma_up_3std']},
        #w20
         {'close' : [  'close_1hour_w20_ma_low_2std', 'close_1hour_w20_ma_up_2std', 'close_1hour_w20_ma_low_3std', 'close_1hour_w20_ma_up_3std']},
         {'volume' : [  'volume_1hour_w20_ma_low_2std', 'volume_1hour_w20_ma_up_2std', 'volume_1hour_w20_ma_low_3std', 'volume_1hour_w20_ma_up_3std']},
         {'tmos_close' : [  'tmos_close_1hour_w20_ma_low_2std', 'tmos_close_1hour_w20_ma_up_2std', 'tmos_close_1hour_w20_ma_low_3std', 'tmos_close_1hour_w20_ma_up_3std']},
        #w30
         {'close' : [  'close_1hour_w30_ma_low_2std', 'close_1hour_w30_ma_up_2std', 'close_1hour_w30_ma_low_3std', 'close_1hour_w30_ma_up_3std']},
         {'tmos_close' : [  'tmos_close_1hour_w30_ma_low_2std', 'tmos_close_1hour_w30_ma_up_2std', 'tmos_close_1hour_w30_ma_low_3std', 'tmos_close_1hour_w30_ma_up_3std']},
        #w60
         {'close' : [  'close_1hour_w60_ma_low_2std', 'close_1hour_w60_ma_up_2std', 'close_1hour_w60_ma_low_3std', 'close_1hour_w60_ma_up_3std']},
         {'tmos_close' : [  'tmos_close_1hour_w60_ma_low_2std', 'tmos_close_1hour_w60_ma_up_2std', 'tmos_close_1hour_w60_ma_low_3std', 'tmos_close_1hour_w60_ma_up_3std']},
        #w120
         {'close' : [  'close_1hour_w120_ma_low_2std', 'close_1hour_w120_ma_up_2std', 'close_1hour_w120_ma_low_3std', 'close_1hour_w120_ma_up_3std']},
         {'tmos_close' : [  'tmos_close_1hour_w120_ma_low_2std', 'tmos_close_1hour_w120_ma_up_2std', 'tmos_close_1hour_w120_ma_low_3std', 'tmos_close_1hour_w120_ma_up_3std']},


        #w5
        ['close_1hour_w5_min', 'close_1hour_w5_max'],
        ['volume_1hour_w5_min', 'volume_1hour_w5_max'],
        ['tmos_close_1hour_w5_min', 'tmos_close_1hour_w5_max'],
        #w10
        ['close_1hour_w10_min', 'close_1hour_w10_max'],
        ['volume_1hour_w10_min', 'volume_1hour_w10_max'],
        ['tmos_close_1hour_w10_min', 'tmos_close_1hour_w10_max'],
        #w20
        ['close_1hour_w20_min', 'close_1hour_w20_max'],
        ['volume_1hour_w20_min', 'volume_1hour_w20_max'],
        ['tmos_close_1hour_w20_min', 'tmos_close_1hour_w20_max'],
        #w30
        ['close_1hour_w30_min', 'close_1hour_w30_max'],
        ['tmos_close_1hour_w30_min', 'tmos_close_1hour_w30_max'],
        #w60
        ['close_1hour_w60_min', 'close_1hour_w60_max'],
        ['tmos_close_1hour_w60_min', 'tmos_close_1hour_w60_max'],
        #w120
        ['close_1hour_w120_min', 'close_1hour_w120_max'],
        ['tmos_close_1hour_w120_min', 'tmos_close_1hour_w120_max'],
]


GROUPS_1DAY= [
        #windows: 
        #close: w1, w5, w10, w20, w30, w60, w120
        #volume: w1, w5, w10, w20
        #tmos_close: w1, w5, w10, w20, w30, w60, w120
         ['close_1day_w1_roc', 'close_1day_w5_alpha', 'close_1day_w10_alpha', 'close_1day_w20_alpha', 'close_1day_w30_alpha', 'close_1day_w60_alpha', 'close_1day_w120_alpha'],
         ['volume_1day_w1_roc', 'volume_1day_w5_alpha', 'volume_1day_w10_alpha', 'volume_1day_w20_alpha'],
         ['tmos_close_1day_w1_roc', 'tmos_close_1day_w5_alpha', 'tmos_close_1day_w10_alpha', 'tmos_close_1day_w20_alpha', 'tmos_close_1day_w30_alpha', 'tmos_close_1day_w60_alpha', 'tmos_close_1day_w120_alpha'],
    
         ['close_1day_w1_roc', 'close_1day_w5_roc', 'close_1day_w10_roc', 'close_1day_w20_roc', 'close_1day_w30_roc', 'close_1day_w60_roc', 'close_1day_w120_roc'],
         ['volume_1day_w1_roc', 'volume_1day_w5_roc', 'volume_1day_w10_roc', 'volume_1day_w20_roc'],
         ['tmos_close_1day_w1_roc', 'tmos_close_1day_w5_roc', 'tmos_close_1day_w10_roc', 'tmos_close_1day_w20_roc', 'tmos_close_1day_w30_roc', 'tmos_close_1day_w60_roc', 'tmos_close_1day_w120_roc'],

         ['close_1day_w5_mean_abs_pct', 'close_1day_w10_mean_abs_pct', 'close_1day_w20_mean_abs_pct', 'close_1day_w30_mean_abs_pct', 'close_1day_w60_mean_abs_pct', 'close_1day_w120_mean_abs_pct'],
         ['volume_1day_w5_mean_abs_pct', 'volume_1day_w10_mean_abs_pct', 'volume_1day_w20_mean_abs_pct'],
         ['tmos_close_1day_w5_mean_abs_pct', 'tmos_close_1day_w10_mean_abs_pct', 'tmos_close_1day_w20_mean_abs_pct', 'tmos_close_1day_w30_mean_abs_pct', 'tmos_close_1day_w60_mean_abs_pct', 'tmos_close_1day_w120_mean_abs_pct'],


         ['close_1day_w5_std', 'close_1day_w10_std', 'close_1day_w20_std', 'close_1day_w30_std', 'close_1day_w60_std', 'close_1day_w120_std'],
         ['volume_1day_w5_std', 'volume_1day_w10_std', 'volume_1day_w20_std'],
         ['tmos_close_1day_w5_std', 'tmos_close_1day_w10_std', 'tmos_close_1day_w20_std', 'tmos_close_1day_w30_std', 'tmos_close_1day_w60_std', 'tmos_close_1day_w120_std'],


         ['close_1day_w5_norm_std', 'close_1day_w10_norm_std', 'close_1day_w20_norm_std', 'close_1day_w30_norm_std', 'close_1day_w60_norm_std', 'close_1day_w120_norm_std'],
         ['volume_1day_w5_norm_std', 'volume_1day_w10_norm_std', 'volume_1day_w20_norm_std'],
         ['tmos_close_1day_w5_norm_std', 'tmos_close_1day_w10_norm_std', 'tmos_close_1day_w20_norm_std', 'tmos_close_1day_w30_norm_std', 'tmos_close_1day_w60_norm_std', 'tmos_close_1day_w120_norm_std'],


         ['close_1day_w5_rsi', 'close_1day_w10_rsi', 'close_1day_w20_rsi', 'close_1day_w30_rsi', 'close_1day_w60_rsi', 'close_1day_w120_rsi'],
         ['volume_1day_w5_rsi', 'volume_1day_w10_rsi', 'volume_1day_w20_rsi'],
         ['tmos_close_1day_w5_rsi', 'tmos_close_1day_w10_rsi', 'tmos_close_1day_w20_rsi', 'tmos_close_1day_w30_rsi', 'tmos_close_1day_w60_rsi', 'tmos_close_1day_w120_rsi'],


         ['close', 'close_1day_w5_ma', 'close_1day_w10_ma', 'close_1day_w20_ma', 'close_1day_w30_ma', 'close_1day_w60_ma', 'close_1day_w120_ma'],
         ['volume', 'volume_1day_w5_ma', 'volume_1day_w10_ma', 'volume_1day_w20_ma'],
         ['tmos_close', 'tmos_close_1day_w5_ma', 'tmos_close_1day_w10_ma', 'tmos_close_1day_w20_ma', 'tmos_close_1day_w30_ma', 'tmos_close_1day_w60_ma', 'tmos_close_1day_w120_ma'],


         ['close', 'close_1day_w5_expma', 'close_1day_w10_expma', 'close_1day_w20_expma', 'close_1day_w30_expma', 'close_1day_w60_expma', 'close_1day_w120_expma'],
         ['volume', 'volume_1day_w5_expma', 'volume_1day_w10_expma', 'volume_1day_w20_expma'],
         ['tmos_close', 'tmos_close_1day_w5_expma', 'tmos_close_1day_w10_expma', 'tmos_close_1day_w20_expma', 'tmos_close_1day_w30_expma', 'tmos_close_1day_w60_expma', 'tmos_close_1day_w120_expma'],


         {'close' : ['close_1day_w5_min', 'close_1day_w10_min', 'close_1day_w20_min', 'close_1day_w30_min', 'close_1day_w60_min', 'close_1day_w120_min']},
         {'volume' : ['volume_1day_w5_min', 'volume_1day_w10_min', 'volume_1day_w20_min']},
         {'tmos_close' : ['tmos_close_1day_w5_min', 'tmos_close_1day_w10_min', 'tmos_close_1day_w20_min', 'tmos_close_1day_w30_min', 'tmos_close_1day_w60_min', 'tmos_close_1day_w120_min']},

    
         {'close' : ['close_1day_w5_max', 'close_1day_w10_max', 'close_1day_w20_max', 'close_1day_w30_max', 'close_1day_w60_max', 'close_1day_w120_max']},
         {'volume' : ['volume_1day_w5_max', 'volume_1day_w10_max', 'volume_1day_w20_max']},
         {'tmos_close' : ['tmos_close_1day_w5_max', 'tmos_close_1day_w10_max', 'tmos_close_1day_w20_max', 'tmos_close_1day_w30_max', 'tmos_close_1day_w60_max', 'tmos_close_1day_w120_max']},



        #w5
         {'close' : [  'close_1day_w5_ma_low_2std', 'close_1day_w5_ma_up_2std', 'close_1day_w5_ma_low_3std', 'close_1day_w5_ma_up_3std']},
         {'volume' : [  'volume_1day_w5_ma_low_2std', 'volume_1day_w5_ma_up_2std', 'volume_1day_w5_ma_low_3std', 'volume_1day_w5_ma_up_3std']},
         {'tmos_close' : [  'tmos_close_1day_w5_ma_low_2std', 'tmos_close_1day_w5_ma_up_2std', 'tmos_close_1day_w5_ma_low_3std', 'tmos_close_1day_w5_ma_up_3std']},
        #w10
         {'close' : [  'close_1day_w10_ma_low_2std', 'close_1day_w10_ma_up_2std', 'close_1day_w10_ma_low_3std', 'close_1day_w10_ma_up_3std']},
         {'volume' : [  'volume_1day_w10_ma_low_2std', 'volume_1day_w10_ma_up_2std', 'volume_1day_w10_ma_low_3std', 'volume_1day_w10_ma_up_3std']},
         {'tmos_close' : [  'tmos_close_1day_w10_ma_low_2std', 'tmos_close_1day_w10_ma_up_2std', 'tmos_close_1day_w10_ma_low_3std', 'tmos_close_1day_w10_ma_up_3std']},
        #w20
         {'close' : [  'close_1day_w20_ma_low_2std', 'close_1day_w20_ma_up_2std', 'close_1day_w20_ma_low_3std', 'close_1day_w20_ma_up_3std']},
         {'volume' : [  'volume_1day_w20_ma_low_2std', 'volume_1day_w20_ma_up_2std', 'volume_1day_w20_ma_low_3std', 'volume_1day_w20_ma_up_3std']},
         {'tmos_close' : [  'tmos_close_1day_w20_ma_low_2std', 'tmos_close_1day_w20_ma_up_2std', 'tmos_close_1day_w20_ma_low_3std', 'tmos_close_1day_w20_ma_up_3std']},
        #w30
         {'close' : [  'close_1day_w30_ma_low_2std', 'close_1day_w30_ma_up_2std', 'close_1day_w30_ma_low_3std', 'close_1day_w30_ma_up_3std']},
         {'tmos_close' : [  'tmos_close_1day_w30_ma_low_2std', 'tmos_close_1day_w30_ma_up_2std', 'tmos_close_1day_w30_ma_low_3std', 'tmos_close_1day_w30_ma_up_3std']},
        #w60
         {'close' : [  'close_1day_w60_ma_low_2std', 'close_1day_w60_ma_up_2std', 'close_1day_w60_ma_low_3std', 'close_1day_w60_ma_up_3std']},
         {'tmos_close' : [  'tmos_close_1day_w60_ma_low_2std', 'tmos_close_1day_w60_ma_up_2std', 'tmos_close_1day_w60_ma_low_3std', 'tmos_close_1day_w60_ma_up_3std']},
        #w120
         {'close' : [  'close_1day_w120_ma_low_2std', 'close_1day_w120_ma_up_2std', 'close_1day_w120_ma_low_3std', 'close_1day_w120_ma_up_3std']},
         {'tmos_close' : [  'tmos_close_1day_w120_ma_low_2std', 'tmos_close_1day_w120_ma_up_2std', 'tmos_close_1day_w120_ma_low_3std', 'tmos_close_1day_w120_ma_up_3std']},


        #w5
        ['close_1day_w5_min', 'close_1day_w5_max'],
        ['volume_1day_w5_min', 'volume_1day_w5_max'],
        ['tmos_close_1day_w5_min', 'tmos_close_1day_w5_max'],
        #w10
        ['close_1day_w10_min', 'close_1day_w10_max'],
        ['volume_1day_w10_min', 'volume_1day_w10_max'],
        ['tmos_close_1day_w10_min', 'tmos_close_1day_w10_max'],
        #w20
        ['close_1day_w20_min', 'close_1day_w20_max'],
        ['volume_1day_w20_min', 'volume_1day_w20_max'],
        ['tmos_close_1day_w20_min', 'tmos_close_1day_w20_max'],
        #w30
        ['close_1day_w30_min', 'close_1day_w30_max'],
        ['tmos_close_1day_w30_min', 'tmos_close_1day_w30_max'],
        #w60
        ['close_1day_w60_min', 'close_1day_w60_max'],
        ['tmos_close_1day_w60_min', 'tmos_close_1day_w60_max'],
        #w120
        ['close_1day_w120_min', 'close_1day_w120_max'],
        ['tmos_close_1day_w120_min', 'tmos_close_1day_w120_max'],
]

# 2. Make dirs

In [11]:
!mkdir data

mkdir: data: File exists


In [12]:
!mkdir data/raw data/preproc data/lgbm data/result

mkdir: data/raw: File exists
mkdir: data/preproc: File exists
mkdir: data/lgbm: File exists
mkdir: data/result: File exists


In [13]:
!mkdir data/raw/15min data/raw/1hour data/raw/1day    data/preproc/15min data/preproc/1hour data/preproc/1day

mkdir: data/raw/15min: File exists
mkdir: data/raw/1hour: File exists
mkdir: data/raw/1day: File exists
mkdir: data/preproc/15min: File exists
mkdir: data/preproc/1hour: File exists
mkdir: data/preproc/1day: File exists


# 2. Get data

In [15]:
def write_logs(text):
    with open("logs.txt", "a") as f:
        f.write(text)

In [16]:
def load_figi():
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'Start loading figi data' +'\n')
    #make dir

    count_tries = 0
    max_count_tries = 5
    flag_data = False
    while (not flag_data) and (count_tries < max_count_tries):
        count_tries += 1
        try:
            with Client(TOKEN) as client:
                instruments: InstrumentsService = client.instruments
                tickers = []
                for method in ["shares", "bonds", "etfs", "currencies", "futures"]:
                    for item in getattr(instruments, method)().instruments:
                        tickers.append(
                            {
                                "name": item.name,
                                "ticker": item.ticker,
                                "class_code": item.class_code,
                                "figi": item.figi,
                                "uid": item.uid,
                                "type": method,
                                "min_price_increment": quotation_to_decimal(
                                    item.min_price_increment
                                ),
                                "scale": 9 - len(str(item.min_price_increment.nano)) + 1,
                                "lot": item.lot,
                                "trading_status": str(
                                    SecurityTradingStatus(item.trading_status).name
                                ),
                                "api_trade_available_flag": item.api_trade_available_flag,
                                "currency": item.currency,
                                "exchange": item.exchange,
                                "buy_available_flag": item.buy_available_flag,
                                "sell_available_flag": item.sell_available_flag,
                                "short_enabled_flag": item.short_enabled_flag,
                                "klong": quotation_to_decimal(item.klong),
                                "kshort": quotation_to_decimal(item.kshort),
                            }
                        )
        
            tickers_df = pd.DataFrame(tickers)
            dump_pkl(tickers_df, './data/data_figi.pkl')
            
            flag_data = True
        except:
            time.sleep(3)

    if not flag_data:
        write_logs(str(pd.Timestamp.now().round('s')) + ' '+ f'Failed to load figi data' +'\n')


    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'End loading figi data' +'\n')

def get_figi(ticker):

        df_figi = load_pkl('./data/data_figi.pkl')

        df_figi = df_figi[df_figi["ticker"] == ticker]
        if ticker == 'T': # таких два тикера T
            df_figi = df_figi[df_figi['currency'] == 'rub']
            
        assert df_figi.shape[0] == 1, f"{ticker}: error figi"

        figi = df_figi["figi"].iloc[0]
        return figi

In [17]:
def get_all_candles(ticker=None, from_=None, to_=None, interval=None):
    
    figi = get_figi(ticker)
    
    with Client(TOKEN) as client:
        
        data = [elem for elem in client.get_all_candles(figi=figi, 
                                      from_=from_-timedelta(hours=3),
                                      to=to_- timedelta(hours=3),
                                     interval=interval
                                    )
               ]
        
    close = np.zeros(len(data))
    volume = np.zeros(len(data))
    times = np.zeros(len(data), dtype='O')
    for i, elem in enumerate(data):
        close[i] = quotation_to_decimal(elem.close)
        volume[i] = elem.volume
        times[i] = elem.time


    df = pd.DataFrame({'time' : times,
                      'close' : close,
                      'volume' : volume,
                    })
    df['time'] += pd.Timedelta(hours=3)
    return df

In [18]:
# %%time

#load_figi()

# ticker = 'SFIN'
# #2024-06-10 
# data = get_all_candles(ticker=ticker,
#                        from_=datetime.datetime(2024, 12, 29, tzinfo=datetime.timezone.utc),
#                        to_=datetime.datetime(2025, 1, 29, tzinfo=datetime.timezone.utc),
#                        interval=CandleInterval.CANDLE_INTERVAL_HOUR
#                       )

# data['ticker'] = ticker
# data

In [19]:
# %%time

#load_figi()


# ticker = 'SFIN'
# #2024-06-10 
# data = get_all_candles(ticker=ticker,
#                        from_=datetime.datetime(2025, 1, 20, tzinfo=datetime.timezone.utc),
#                        to_=datetime.datetime(2025, 1, 29, tzinfo=datetime.timezone.utc),
#                        interval=CandleInterval.CANDLE_INTERVAL_DAY
#                       )

# data['ticker'] = ticker
# data

In [20]:
def get_data_1day(stocks):
    #loading figi
    load_figi()

    
    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'Start loading daily data' +'\n')    
    
    #make dir
    time_name = str(pd.Timestamp.now().date()) 
    if not os.path.exists(f'./data/raw/1day/{time_name}'):
        os.mkdir(f'./data/raw/1day/{time_name}')

    #loading data
    end = pd.Timestamp.now()+pd.Timedelta(days=1) #+1day, чтобы выгрузить все, что есть (+1 точка для проверки актуальности данных)
    start = end - pd.Timedelta(days=240)
    
    for ticker in tqdm(stocks+['TMOS']): 
        
        count_tries = 0
        max_count_tries = 4
        flag_data = False
        while (not flag_data) and (count_tries < max_count_tries):
            count_tries += 1
            try:
                #spec case of X5
                if ticker in ['X5', 'TMOS']:
                    data = get_all_candles(ticker=ticker,
                                   from_=start - pd.Timedelta(days=9*30),
                                   to_=end,
                                   interval=CandleInterval.CANDLE_INTERVAL_DAY)
                else:
                    data = get_all_candles(ticker=ticker,
                                   from_=start,
                                   to_=end,
                                   interval=CandleInterval.CANDLE_INTERVAL_DAY)

                flag_data = True
            except:
                time.sleep(3)    
        #saving
        if flag_data:
            data = data[['time', 'close', 'volume']]
            data['ticker'] = ticker
            dump_pkl(data, f'./data/raw/1day/{time_name}/{ticker}.pkl')
        else:
            write_logs(str(pd.Timestamp.now().round('s')) + ' '+ f'{ticker}: failed to load daily data' +'\n')
            
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'End loading daily data' +'\n')





def get_data_1hour(stocks):
    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'Start loading hourly data' +'\n')

    #make dir
    time_name = str(pd.Timestamp.now()).split(':')[0].replace(' ', '_') #date+hour
    if not os.path.exists(f'./data/raw/1hour/{time_name}'):
        os.mkdir(f'./data/raw/1hour/{time_name}')
    
    #loading data
    end = pd.Timestamp.now()+pd.Timedelta(days=1) #+1day, чтобы выгрузить все, что есть (+1 точка для проверки целостности данных)
    start = end - pd.Timedelta(days=25)
    for ticker in tqdm(stocks+['TMOS']): 
        
        count_tries = 0
        max_count_tries = 4
        flag_data = False
        while (not flag_data) and (count_tries < max_count_tries):
            count_tries += 1
            try:
                data = get_all_candles(ticker=ticker,
                               from_=start,
                               to_=end,
                               interval=CandleInterval.CANDLE_INTERVAL_HOUR)
                flag_data = True
            except:
                time.sleep(3)    
        #saving
        if flag_data:
            data = data[['time', 'close', 'volume']]
            data['ticker'] = ticker
            dump_pkl(data, f'./data/raw/1hour/{time_name}/{ticker}.pkl')
        else:
            write_logs(str(pd.Timestamp.now().round('s')) + ' '+ f'{ticker}: failed to load hourly data' +'\n')

    
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'End loading hourly data' +'\n')






def get_data_15min(stocks):
    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'Start loading 15min data' +'\n')

    #make dir
    time_name = f"{str(pd.Timestamp.now()).split(':')[0].replace(' ', '_')}_{(pd.Timestamp.now().minute // 15) * 15}"  #date+hour+15min
    if not os.path.exists(f'./data/raw/15min/{time_name}'):
        os.mkdir(f'./data/raw/15min/{time_name}')
    
    #loading data
    end = pd.Timestamp.now()+pd.Timedelta(days=1) #+1day, чтобы выгрузить все, что есть (+1 точка для проверки целостности данных)
    start = end - pd.Timedelta(days=8)
    for ticker in tqdm(stocks+['TMOS']): 
        
        count_tries = 0
        max_count_tries = 4
        flag_data = False
        while (not flag_data) and (count_tries < max_count_tries):
            count_tries += 1
            try:
                data = get_all_candles(ticker=ticker,
                               from_=start,
                               to_=end,
                               interval=CandleInterval.CANDLE_INTERVAL_15_MIN)
                flag_data = True
            except:
                time.sleep(3)    
        #saving
        if flag_data:
            data = data[['time', 'close', 'volume']]
            data['ticker'] = ticker
            dump_pkl(data, f'./data/raw/15min/{time_name}/{ticker}.pkl')
        else:
            write_logs(str(pd.Timestamp.now().round('s')) + ' '+ f'{ticker}: failed to load 15min data' +'\n')

    
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'End loading 15min data' +'\n')
    

In [21]:
# get_data_15min(STOCKS)

In [22]:
# get_data_1hour(STOCKS)

In [23]:
# get_data_1day(STOCKS)

In [24]:
# df = load_pkl('./data/raw/15min/2025-02-14_15_0/NLMK.pkl')
# df

In [25]:
# df = load_pkl('./data/raw/1hour/2025-02-14_15/NLMK.pkl')
# df

In [26]:
# df = load_pkl('./data/raw/1day/2025-02-14/X5.pkl')
# df

# 4. Preproc data

In [28]:
def is_business_day(date):
    return bool(len(pd.bdate_range(date, date)))

def time_cut_data_1day(data):
    data['time'] = data['time'].apply(lambda x: x.tz_convert(None))
    
    mask_bd = np.array(data['time'].apply(lambda x: is_business_day(x)))
    data = data.loc[mask_bd, :]
    data.reset_index(drop=True, inplace=True)

    return data

def preproc_data_1day(stocks):
    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'Start preproc daily data' +'\n')

    #make dir
    time_name = str(pd.Timestamp.now().date()) 
    if not os.path.exists(f'./data/preproc/1day/{time_name}'):
        os.mkdir(f'./data/preproc/1day/{time_name}')

    #tmos
    if not os.path.exists(f"./data/raw/1day/{time_name}/TMOS.pkl"):
        write_logs(str(pd.Timestamp.now().round('s')) + ' '+ f'TMOS: failed to preproc daily data' +'\n')
        return
    data_tmos = load_pkl(f"./data/raw/1day/{time_name}/TMOS.pkl")

    #preproc
    for ticker in tqdm(stocks):
        try:
            data = load_pkl(f"./data/raw/1day/{time_name}/{ticker}.pkl")
        
            data_ext = data.merge(data_tmos[['time', 'close']].copy().rename(columns={"close" : "tmos_close"}), how='left', on='time')
            assert data.shape[0] == data_ext.shape[0], 'Error join tmos'
            
            data = time_cut_data_1day(data_ext)
            assert data['tmos_close'].isnull().sum() == 0, 'Error tmos_close nulls' #праздники, в них индекс не торгуется (мб в послед время рассчитывется?)
            #data['tmos_close'] = data['tmos_close'].ffill()

            #Actual time date (не самая идеальная реализация, так как чекает наличие следующей точки после прогнозной, а не прогнозную)
            last_time = data['time'].iloc[-1]
            assert last_time.date() == pd.Timestamp.now().date(), 'Error time_data' 
            
            #Обрежем последнюю точку 
            mask = data['time'] < pd.Timestamp.now().floor('d')
            data = data[mask]
            
            data = data.iloc[-(COUNT_POINTS+1):]
            data.reset_index(drop=True, inplace=True)

            #len
            assert data.shape[0] == (COUNT_POINTS+1), 'Error len'
            
            #Заполняемость
            diff_time_day = data['time'].diff() / pd.Timedelta(hours=24)
            assert (diff_time_day.iloc[1:] % 1 == 0).all(), 'Error filling 1'
            assert (diff_time_day.iloc[1:] == 2).sum() < 5, 'Error filling 2'
            # assert (diff_time_day.iloc[1:] < 10).all(), 'Error 5' #переезд акций ломает тест

            #duplicates in time
            assert (diff_time_day != 0).all(), 'Error time duplicates'

            #sort in time
            assert (diff_time_day.iloc[1:] > 0).all(), 'Error time sort'

            #Nulls
            assert not data.isnull().any().any(), 'Error nulls'  
            
            dump_pkl(data, f'./data/preproc/1day/{time_name}/{ticker}.pkl')
        except Exception as exception:
            write_logs(str(pd.Timestamp.now().round('s')) + ' '+ f'{ticker}: failed to preproc daily data ({exception})' +'\n')

    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'End preproc daily data' +'\n')






def time_cut_data_1hour_15min(data):
    data['time'] = data['time'].apply(lambda x: x.tz_convert(None))
    
    mask_bd = np.array(data['time'].apply(lambda x: is_business_day(x)))
    data = data.loc[mask_bd, :]

    mask_volume = np.array(datetime.time(10, 0) <= pd.to_datetime(data['time'], format='%H:%M').dt.time)
    data = data.loc[mask_volume, :]
    data.reset_index(drop=True, inplace=True)
    
    return data

def preproc_data_1hour(stocks):
    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'Start preproc hourly data' +'\n')

    #make dir
    time_name = str(pd.Timestamp.now()).split(':')[0].replace(' ', '_') #date+hour
    if not os.path.exists(f'./data/preproc/1hour/{time_name}'):
        os.mkdir(f'./data/preproc/1hour/{time_name}')

    #tmos
    if not os.path.exists(f"./data/raw/1hour/{time_name}/TMOS.pkl"):
        write_logs(str(pd.Timestamp.now().round('s')) + ' '+ f'TMOS: failed to preproc hourly data' +'\n')
        return
    data_tmos = load_pkl(f"./data/raw/1hour/{time_name}/TMOS.pkl")
    #костыль
    if data_tmos['time'].iloc[-1] == data_tmos['time'].iloc[-2]:
        data_tmos = data_tmos.iloc[:-1].reset_index(drop=True)

    #preproc
    for ticker in tqdm(stocks):
        try:
            data = load_pkl(f"./data/raw/1hour/{time_name}/{ticker}.pkl")
        
            data_ext = data.merge(data_tmos[['time', 'close']].copy().rename(columns={"close" : "tmos_close"}), how='left', on='time')
            assert data.shape[0] == data_ext.shape[0], 'Error join tmos'
        
            data = time_cut_data_1hour_15min(data_ext)
            assert data['tmos_close'].isnull().sum() <= 5, 'Error tmos_close nulls' #Мосбиржа стоит (остаовка торгов), а акции на СПБ бирже торгуются (LSRG, SPBE, ...)
            #костыль, но в рамках разумного, взял 5 - как кол-во часов в вечерней сессии
            data['tmos_close'] = data['tmos_close'].ffill()

            #Actual time date (не самая идеальная реализация, так как чекает наличие следующую точку после прогнозной, а не прогнозную)
            last_time = data['time'].iloc[-1]
            #округление времени до 1часа
            cur_round_time = pd.Timestamp.now().floor('h')
            assert (last_time == cur_round_time) or (last_time == (cur_round_time - pd.Timedelta(hours=1))), 'Error time_data' #послабление для малоликвидных акций
            
            
            #Обрежем последнюю точку (или две, они бывают задваиваются - особенность онлайн данных)
            mask = data['time'] < cur_round_time
            data = data[mask]
            
            data = data.iloc[-(COUNT_POINTS+1):]
            data.reset_index(drop=True, inplace=True)

            #len
            assert data.shape[0] == (COUNT_POINTS+1), 'Error len'
            
            #Заполняемость
            diff_time_hour = data['time'].diff() / pd.Timedelta(hours=1)
            assert (diff_time_hour.iloc[1:] % 1 == 0).all(), 'Error filling 1'
            assert (diff_time_hour.iloc[1:] == 2).sum() < 10, 'Error filling 2'
            assert (diff_time_hour.iloc[1:] < 24*4).all(), 'Error filling 3' #допускает до 4 дней выходных

            #duplicates in time
            assert (diff_time_hour != 0).all(), 'Error time duplicates'

            #sort in time
            assert (diff_time_hour.iloc[1:] > 0).all(), 'Error time sort'
            
            #Nulls
            assert not data.isnull().any().any(), 'Error nulls'
    
        
            dump_pkl(data, f'./data/preproc/1hour/{time_name}/{ticker}.pkl')
        except Exception as exception:
            write_logs(str(pd.Timestamp.now().round('s')) + ' '+ f'{ticker}: failed to preproc hourly data ({exception})' +'\n')

    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'End preproc hourly data' +'\n')




def preproc_data_15min(stocks):
    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'Start preproc 15min data' +'\n')

    #make dir
    time_name = f"{str(pd.Timestamp.now()).split(':')[0].replace(' ', '_')}_{(pd.Timestamp.now().minute // 15) * 15}"  #date+hour+15min
    if not os.path.exists(f'./data/preproc/15min/{time_name}'):
        os.mkdir(f'./data/preproc/15min/{time_name}')

    #tmos
    if not os.path.exists(f"./data/raw/15min/{time_name}/TMOS.pkl"):
        write_logs(str(pd.Timestamp.now().round('s')) + ' '+ f'TMOS: failed to preproc 15min data' +'\n')
        return
    data_tmos = load_pkl(f"./data/raw/15min/{time_name}/TMOS.pkl")
    #костыль
    if data_tmos['time'].iloc[-1] == data_tmos['time'].iloc[-2]:
        data_tmos = data_tmos.iloc[:-1].reset_index(drop=True)

    #preproc
    for ticker in tqdm(stocks):
        try:
            data = load_pkl(f"./data/raw/15min/{time_name}/{ticker}.pkl")
        
            data_ext = data.merge(data_tmos[['time', 'close']].copy().rename(columns={"close" : "tmos_close"}), how='left', on='time')
            assert data.shape[0] == data_ext.shape[0], 'Error join tmos'
        
            data = time_cut_data_1hour_15min(data_ext)
            assert data['tmos_close'].isnull().sum()  < 20, 'Error tmos_close nulls' #Мосбиржа стоит (остановка торгов), а акции на СПБ бирже торгуются (LSRG, SPBE, ...)
            #костыль, но в рамках разумного
            data['tmos_close'] = data['tmos_close'].ffill()

            #Actual time date (не самая идеальная реализация, так как чекает наличие следующую точку после прогнозной, а не прогнозную)
            last_time = data['time'].iloc[-1]
            #округление времени до 15мин
            cur_round_time = pd.Timestamp(f"{str(pd.Timestamp.now()).split(':')[0]}:{(pd.Timestamp.now().minute // 15) * 15}")
            assert (last_time == cur_round_time) or (last_time == (cur_round_time -  pd.Timedelta(minutes=15))), 'Error time_data'#послабление для малоликвидных акций
            
            #Обрежем последнюю точку (или две, они бывают задваиваются - особенность онлайн данных)
            mask = data['time'] < cur_round_time
            data = data[mask]
            
            data = data.iloc[-(COUNT_POINTS+1):]
            data.reset_index(drop=True, inplace=True)

            #len
            assert data.shape[0] == (COUNT_POINTS+1), 'Error len'
            
            #Заполняемость
            diff_time_15min = data['time'].diff() / pd.Timedelta(minutes=15)
            assert (diff_time_15min.iloc[1:] % 1 == 0).all(), 'Error filling 1'
            assert (diff_time_15min.iloc[1:] == 2).sum() < 10, 'Error filling 2'
            assert (diff_time_15min.iloc[1:] < 5*24*4).all(), 'Error filling 3'

            #duplicates in time
            assert (diff_time_15min != 0).all(), 'Error time duplicates'

            #sort in time
            assert (diff_time_15min.iloc[1:] > 0).all(), 'Error time sort'
            
            #Nulls
            assert not data.isnull().any().any(), 'Error nulls'
    
        
            dump_pkl(data, f'./data/preproc/15min/{time_name}/{ticker}.pkl')
        except Exception as exception:
            write_logs(str(pd.Timestamp.now().round('s')) + ' '+ f'{ticker}: failed to preproc 15min data ({exception})' +'\n')

    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'End preproc 15min data' +'\n')









In [29]:
# preproc_data_1day(STOCKS)

In [30]:
# preproc_data_1hour(STOCKS)

In [31]:
# preproc_data_15min(STOCKS)

In [32]:
# data = load_pkl('./data/preproc/1day/2025-02-14/YDEX.pkl')
# data

In [33]:
# data = load_pkl('./data/preproc/1hour/2025-02-14_15/YDEX.pkl')
# data

In [34]:
# data = load_pkl('./data/preproc/15min/2025-02-14_15_45//YDEX.pkl')
# data

# 5. Prepare data to lgbm

In [36]:
from sklearn.linear_model import LinearRegression

def calculate_exp_ma(data, window):
    return data.ewm(span=window, min_periods=window).mean().squeeze().values


def calculate_bollinger_bands(data, window):
    """Calculate Bollinger Bands"""
    rolling_mean = data.rolling(window=window, min_periods=1).mean().values
    rolling_std = data.rolling(window=window, min_periods=1).std().values

    norm_rolling_std = rolling_std/rolling_mean

    num_of_std = 2
    lower_band_2std = rolling_mean - (rolling_std * num_of_std)
    upper_band_2std = rolling_mean + (rolling_std * num_of_std)
    
    num_of_std = 3
    lower_band_3std = rolling_mean - (rolling_std * num_of_std)
    upper_band_3std = rolling_mean + (rolling_std * num_of_std)
    
    
    return rolling_mean, rolling_std, norm_rolling_std, lower_band_2std, upper_band_2std, lower_band_3std, upper_band_3std

def calculate_rsi(data, window):
    """Calculate Relative Strength Index"""
    delta = data.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    mask = avg_loss == 0
    rsi[mask] = 100
    
    return rsi.values

def calculate_roc(data, periods):
    """Calculate Rate of Change."""
    roc = ((data - data.shift(periods)) / data.shift(periods))
    return roc.values



def calc_stats(data, window=None, feat_name=None):
    #mean, std
    rolling_mean, rolling_std, norm_rolling_std,\
    lower_band_2std, upper_band_2std, lower_band_3std, upper_band_3std = calculate_bollinger_bands(data, window)

    #mean_abs_pct
    mean_abs_pct = data.pct_change(periods=1).rolling(window=window, min_periods=1).apply(lambda x: x.abs().mean())
        
    #alpha
    alpha = data.rolling(window=window, min_periods=2).apply(lambda x: LinearRegression().fit(x.values.reshape(-1, 1), np.arange(x.shape[0])).coef_[0])

    #min, max
    rolling_min = data.rolling(window=window, min_periods=1).min().values
    rolling_max = data.rolling(window=window, min_periods=1).max().values
    
    #rsi
    rsi = calculate_rsi(data, window)
    
    #roc
    roc = calculate_roc(data, window)
    diff = data.diff(window).values

    #exp_ma
    exp_ma = calculate_exp_ma(data, window)
    
    df_features = pd.DataFrame({f'{feat_name}_ma' : rolling_mean,
                        f'{feat_name}_std' : rolling_std,
                        f'{feat_name}_norm_std' : norm_rolling_std,
                        f'{feat_name}_ma_low_2std' : lower_band_2std,
                        f'{feat_name}_ma_up_2std' : upper_band_2std,
                        f'{feat_name}_ma_low_3std' : lower_band_3std,
                        f'{feat_name}_ma_up_3std' : upper_band_3std, 

                        f'{feat_name}_mean_abs_pct' : mean_abs_pct,
                            
                        f'{feat_name}_alpha' : alpha,
                            
                        f'{feat_name}_min' : rolling_min,
                        f'{feat_name}_max' : rolling_max,
                        f'{feat_name}_rsi' : rsi,
                        f'{feat_name}_roc' : roc,
                        f'{feat_name}_diff' : diff,
                        f'{feat_name}_expma' : exp_ma,
                        })
    return df_features


def calc_stats_diff_1(data, feat_name=None):
    return pd.DataFrame({f'{feat_name}_roc' : data.pct_change(periods=1).values,
                        f'{feat_name}_diff' : data.diff(1).values,
                        })

def calc_levels(data, window=None, levels=None, feat_name=None):
    
    #уровни
    data_levels = []
    column_names = []
    for i in range(1, len(levels)):
        level_low = levels[i-1]
        level_high = levels[i]
        data_levels += [data.rolling(window=window, min_periods=1).apply(lambda x: (((1+level_low)*x.values[-1] < x.values) & (x.values <= (1+level_high)*x.values[-1])).sum()).values]
        data_levels += [data.rolling(window=window, min_periods=1).apply(lambda x: (((1-level_high)*x.values[-1] <= x.values) & (x.values < (1-level_low)*x.values[-1])).sum()).values]

        column_names += [f"{feat_name}_lvl_{1+level_low}-{1+level_high}"]
        column_names += [f"{feat_name}_lvl_-{1-level_high}-{1-level_low}"]
    df_levels = pd.DataFrame({column_names[i]:data_levels[i] for i in range(len(column_names))})
    return df_levels


In [37]:
def calculate_features(df_ticker, postfix=None):
    dfs = [df_ticker]
    
    levels =      [0, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.07]
    levels_tmos = [0, 0.005, 0.01, 0.015, 0.02, 0.03, 0.04, 0.05]


    #w1
    df_close = calc_stats_diff_1(df_ticker[f'close{postfix}'], feat_name=f'close{postfix}_w1')
    df_volume = calc_stats_diff_1(df_ticker[f'volume{postfix}'], feat_name=f'volume{postfix}_w1')
    df_tmos_close = calc_stats_diff_1(df_ticker[f'tmos_close{postfix}'], feat_name=f'tmos_close{postfix}_w1')
    assert df_ticker.shape[0] == df_close.shape[0] == df_volume.shape[0] == df_tmos_close.shape[0], 'Error w1'
    dfs += [df_close.copy(), df_volume.copy(), df_tmos_close.copy()]

    #w5
    window = 5
    df_close = calc_stats(df_ticker[f'close{postfix}'], window=window, feat_name=f'close{postfix}_w{window}')
    df_volume = calc_stats(df_ticker[f'volume{postfix}'], window=window, feat_name=f'volume{postfix}_w{window}')
    df_tmos_close = calc_stats(df_ticker[f'tmos_close{postfix}'], window=window, feat_name=f'tmos_close{postfix}_w{window}')
    assert df_ticker.shape[0] == df_close.shape[0] == df_volume.shape[0] == df_tmos_close.shape[0], f'Error w{window}'
    dfs += [df_close.copy(), df_volume.copy(), df_tmos_close.copy()]
    
    #w10
    window = 10
    df_close = calc_stats(df_ticker[f'close{postfix}'], window=window, feat_name=f'close{postfix}_w{window}')
    df_volume = calc_stats(df_ticker[f'volume{postfix}'], window=window, feat_name=f'volume{postfix}_w{window}')
    df_tmos_close = calc_stats(df_ticker[f'tmos_close{postfix}'], window=window, feat_name=f'tmos_close{postfix}_w{window}')
    assert df_ticker.shape[0] == df_close.shape[0] == df_volume.shape[0] == df_tmos_close.shape[0], f'Error w{window}'
    dfs += [df_close.copy(), df_volume.copy(), df_tmos_close.copy()]
    
    #w20
    window = 20
    df_close = calc_stats(df_ticker[f'close{postfix}'], window=window, feat_name=f'close{postfix}_w{window}')
    df_volume = calc_stats(df_ticker[f'volume{postfix}'], window=window, feat_name=f'volume{postfix}_w{window}')
    df_tmos_close = calc_stats(df_ticker[f'tmos_close{postfix}'], window=window, feat_name=f'tmos_close{postfix}_w{window}')
    assert df_ticker.shape[0] == df_close.shape[0] == df_volume.shape[0] == df_tmos_close.shape[0], f'Error w{window}'
    dfs += [df_close.copy(), df_volume.copy(), df_tmos_close.copy()]
    
    #w30
    window = 30
    df_close = calc_stats(df_ticker[f'close{postfix}'], window=window, feat_name=f'close{postfix}_w{window}')
    #df_volume = calc_stats(df_ticker[f'volume{postfix}'], window=window, feat_name=f'volume{postfix}_w{window}')
    df_tmos_close = calc_stats(df_ticker[f'tmos_close{postfix}'], window=window, feat_name=f'tmos_close{postfix}_w{window}')
    df_close_levels = calc_levels(df_ticker[f'close{postfix}'], window=window, levels=levels, feat_name=f'close{postfix}_w{window}')
    df_tmos_close_levels = calc_levels(df_ticker[f'tmos_close{postfix}'], window=window, levels=levels_tmos, feat_name=f'tmos_close{postfix}_w{window}')
    assert df_ticker.shape[0] == df_close.shape[0] == df_tmos_close.shape[0] == df_close_levels.shape[0] == df_tmos_close_levels.shape[0], f'Error w{window}'
    dfs += [df_close.copy(), df_tmos_close.copy(), df_close_levels.copy(), df_tmos_close_levels.copy()]
    
    #w60
    window = 60
    df_close = calc_stats(df_ticker[f'close{postfix}'], window=window, feat_name=f'close{postfix}_w{window}')
    #df_volume = calc_stats(df_ticker[f'volume{postfix}'], window=window, feat_name=f'volume{postfix}_w{window}')
    df_tmos_close = calc_stats(df_ticker[f'tmos_close{postfix}'], window=window, feat_name=f'tmos_close{postfix}_w{window}')
    assert df_ticker.shape[0] == df_close.shape[0] == df_tmos_close.shape[0], f'Error w{window}'
    dfs += [df_close.copy(), df_tmos_close.copy()]
    
    #w120
    window = 120
    df_close = calc_stats(df_ticker[f'close{postfix}'], window=window, feat_name=f'close{postfix}_w{window}')
    #df_volume = calc_stats(df_ticker[f'volume{postfix}'], window=window, feat_name=f'volume{postfix}_w{window}')
    df_tmos_close = calc_stats(df_ticker[f'tmos_close{postfix}'], window=window, feat_name=f'tmos_close{postfix}_w{window}')
    df_close_levels = calc_levels(df_ticker[f'close{postfix}'], window=window, levels=levels, feat_name=f'close{postfix}_w{window}')
    df_tmos_close_levels = calc_levels(df_ticker[f'tmos_close{postfix}'], window=window, levels=levels_tmos, feat_name=f'tmos_close{postfix}_w{window}')
    assert df_ticker.shape[0] == df_close.shape[0] == df_tmos_close.shape[0] == df_close_levels.shape[0] == df_tmos_close_levels.shape[0], f'Error w{window}'
    dfs += [df_close.copy(), df_tmos_close.copy(), df_close_levels.copy(), df_tmos_close_levels.copy()]

   
    df = pd.concat(dfs, axis=1)
    assert (df_ticker.shape[0] == df.shape[0]) and (df.shape[1] == sum([elem.shape[1] for elem in dfs])), 'Error concat'

    return df


In [38]:
def uniq_pairs(cols):
    pairs = []
    for i in range(len(cols)-1):
        for j in range(i+1, len(cols)):
            pairs += [(cols[i], cols[j])]
    return pairs

def calc_relative_features(df, groups):
    for group in groups:
        if type(group) == list:
            pairs = uniq_pairs(group)
            for pair in pairs:
                new_col = f'{pair[0]}/{pair[1]}'
                df[new_col] = df[pair[0]] / (df[pair[1]] + np.finfo(np.float32).eps)

        if type(group) == dict:
            pair1 = list(group.keys())[0]
            for pair0 in group[pair1]:
                new_col = f'{pair0}/{pair1}'
                df[new_col] = df[pair0] / (df[pair1] + np.finfo(np.float32).eps)

    return df


In [39]:
def prepare_data_lgbm(stocks, features):
    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'Start prepare data to lgbm' +'\n')

    name_day = str(pd.Timestamp.now().date()) #date
    name_hour = str(pd.Timestamp.now()).split(':')[0].replace(' ', '_') #date+hour
    name_15min = f"{str(pd.Timestamp.now()).split(':')[0].replace(' ', '_')}_{(pd.Timestamp.now().minute // 15) * 15}"  #date+hour+15min

    #prepare
    data_lgbm = []

    #тут можно ускорить в 120 раз, если считать только последнюю точку в каждом df (можно еще считать только нужные фичи - но это перебор, плохо масштабируется)
    for ticker in tqdm(stocks):
        try:
            df_1day = load_pkl(f"./data/preproc/1day/{name_day}/{ticker}.pkl")
            df_1hour = load_pkl(f"./data/preproc/1hour/{name_hour}/{ticker}.pkl")
            df_15min = load_pkl(f"./data/preproc/15min/{name_15min}/{ticker}.pkl")

            df_1day = df_1day.rename(columns={col : col+'_1day' for col in df_1day.columns})
            df_1hour = df_1hour.rename(columns={col : col+'_1hour' for col in df_1hour.columns})

            # вот тут
            df_1day = calculate_features(df_1day, postfix='_1day').iloc[-1:].reset_index(drop=True)
            df_1hour = calculate_features(df_1hour, postfix='_1hour').iloc[-1:].reset_index(drop=True)
            df_15min = calculate_features(df_15min, postfix='').iloc[-1:].reset_index(drop=True)

            #time features
            df_15min['hour'] = df_15min['time'].dt.hour
            df_15min['day'] = df_15min['time'].dt.day
            df_15min['weekday'] = np.minimum(df_15min['time'].dt.dayofweek, 4) / 4

            assert not df_1day.isnull().any().any(), 'Error nulls df_1day'
            assert not df_1hour.isnull().any().any(), 'Error nulls df_1hour'
            assert not df_15min.isnull().any().any(), 'Error nulls df_15min'
            
            data = pd.concat([df_15min, df_1hour, df_1day], axis=1)
        
            data_lgbm += [data.copy()]
            
        except Exception as exception:
            write_logs(str(pd.Timestamp.now().round('s')) + ' '+ f'{ticker}: failed to prepare data to lgbm ({exception})' +'\n')

    try:
        data_lgbm = pd.concat(data_lgbm)
        data_lgbm = calc_relative_features(data_lgbm, GROUPS_1DAY+GROUPS_1HOUR+GROUPS_15MIN)
        
        assert not data_lgbm.isnull().any().any(), 'Error nulls data_lgbm'  
        
        data_lgbm = data_lgbm[['time', 'ticker', 'close', 'time_1hour', 'time_1day'] + features].reset_index(drop=True).copy()
        dump_pkl(data_lgbm, f"./data/lgbm/data_lgbm_{name_15min}.pkl")

    except Exception as exception:
        write_logs(str(pd.Timestamp.now().round('s')) + ' '+ f'Failed to prepare data to lgbm ({exception})' +'\n')

    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'End prepare data to lgbm' +'\n')

In [40]:
# prepare_data_lgbm(STOCKS, FEATURES)

In [41]:
#df = load_pkl('./data/lgbm/data_lgbm_2025-02-04_16.pkl')

# 6. predict lgbm

In [43]:
class lgbm_model:
    def __init__(self, name=None, strategy=None, threshold=None, path=None, features=None):
        self.name = name
        self.strategy = strategy
        self.threshold = threshold
        self.model = load_pkl(path)
        self.features = features
        
    def predict(self, data_cp):
        data = data_cp.copy()

        data['model'] = self.name
        
        data['y_pred'] = self.model.predict(data[self.features])
        data['y_pred_bin'] = (data['y_pred'].values >= self.threshold).astype(int)
    
        #можно для больших чисел без 6 знаков после запятой сделать
        data['fix_lose'] = np.round(data['close'] * self.strategy[0], 6)
        data['fix_win'] = np.round(data['close'] * self.strategy[1], 6)
    
        data = data[['time', 'ticker', 'close', 'model', 'y_pred', 'y_pred_bin', 'fix_lose', 'fix_win']]

        return data
        

In [44]:
# MODEL1 = lgbm_model(name='short_+0.5_-1.5_4hour', 
#                     strategy=(1.005, 0.985), 
#                     threshold=0.16409330252458745, #99
#                     path='../bst_mdl/model_+0.5_-1.5_4hour/model_+0.5_-1.5_4hour.pkl', 
#                     features=MODEL_FEATURES1)

MODEL2 = lgbm_model(name='short_+0.5_-1.5_4hour_v2', 
                    strategy=(1.005, 0.985), 
                    threshold=0.3095578175723138, #995
                    path='../bst_mdl/model_+0.5_-1.5_4hour_v2/model_+0.5_-1.5_4hour.pkl', 
                    features=MODEL_FEATURES1)


MODEL3 = lgbm_model(name='long_-0.5_+1.5_4hour', 
                    strategy=(0.995, 1.015), 
                    threshold=0.2591240486482249, #99
                    path='../bst_mdl/model_-0.5_+1.5_4hour/model_-0.5_+1.5_4hour.pkl', 
                    features=MODEL_FEATURES2)



MODELS = [#MODEL1,
    MODEL2,
    MODEL3]

In [45]:
def predict_lgbm(models):
    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'Start lgbm predicting' +'\n')

    try:
        name_15min = f"{str(pd.Timestamp.now()).split(':')[0].replace(' ', '_')}_{(pd.Timestamp.now().minute // 15) * 15}"  #date+hour+15min
        data = load_pkl(f"./data/lgbm/data_lgbm_{name_15min}.pkl")

        data_result = []
        for model in models:
            data_result += [model.predict(data)]
            
        data_result = pd.concat(data_result).reset_index(drop=True)
        
        dump_pkl(data_result, f"./data/result/data_result_{name_15min}.pkl")
    except Exception as exception:
        write_logs(str(pd.Timestamp.now().round('s')) + ' ' + f'Failed lgbm predicting ({exception})' +'\n')

    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'End lgbm predicting' +'\n')

In [46]:
# predict_lgbm(MODELS)

In [47]:
# df = load_pkl('./data/result/data_result_2025-02-14_16_15.pkl')
# df

# 7. Notification in telegram

In [49]:
import re
def escape_markdown(text):
    """функция для экранирования символов перед отправкой в маркдауне телеграма"""
    pattern = r"([_*\[\]()~|`])"
    return re.sub(pattern, r"\\\1", text)

def telegram_bot_sendtext(bot_message):
    #bot_message = escape_markdown(bot_message)
    send_text = 'https://api.telegram.org/bot' + BOT_TOKEN + '/sendMessage?chat_id=' + BOT_CHAT_ID + '&parse_mode=Markdown&text=' + bot_message
    response = requests.get(send_text)
    return response.json()


# test = telegram_bot_sendtext("Testing Telegram bot")
# print(test)

In [50]:
#load_pkl('./data/result/data_result_2025-02-04_16.pkl')

In [51]:
def send_notification_tg(stocks):    
    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'Start sending msg' +'\n')
    
    try:
        name_15min = f"{str(pd.Timestamp.now()).split(':')[0].replace(' ', '_')}_{(pd.Timestamp.now().minute // 15) * 15}"  #date+hour+15min
        data = load_pkl(f"./data/result/data_result_{name_15min}.pkl")

        msg = f'*signals:* {data['y_pred_bin'].sum()}\n'

        for model in data['model'].unique():
            msg += f"\n*{model}*\n\n"
            data_model = data[data['model'] == model]
            for i, row in data_model.iterrows():
                if row['y_pred_bin'] == 1:
                    msg += f"{row['ticker']}, close = {row['close']}\n"
                    sign = '<' if row['fix_lose'] < row['fix_win'] else '>'
                    msg += f"{row['fix_lose']} {sign} {row['fix_win']}\n\n"

        msg += '\n*no data:*\n'
        no_data_stocks = set(stocks).difference(set(data['ticker'].tolist()))
        msg += "\n".join(list(no_data_stocks))
        
        _ = telegram_bot_sendtext(msg)
              
    except Exception as exception:
        write_logs(str(pd.Timestamp.now().round('s')) + ' ' + f'Failed sending msg ({exception})' +'\n')

    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'End sending msg' +'\n')
    

In [52]:
# send_notification_tg(STOCKS)

# x. Daemon Khibiny

In [54]:
def is_business_day(date):
    return bool(len(pd.bdate_range(date, date)))

def flag_work_1day(date):
    name_day = str(pd.Timestamp.now().date()) #date
    if (date.hour >= 10) and (date.minute >= 0) and is_business_day(date) and (not os.path.exists(f'./data/raw/1day/{name_day}')):
        return True
    if (date.hour == 10) and (date.minute == 10) and is_business_day(date): #дополнительное скачивание на всякий случай, чтобы не терять целый день в худшем случае
        return True
    return False
    
def flag_work_1hour(date):
    name_hour = str(pd.Timestamp.now()).split(':')[0].replace(' ', '_') #date+hour
    if (date.hour >= 10) and (date.minute >= 0) and is_business_day(date) and (not os.path.exists(f'./data/raw/1hour/{name_hour}')):
        return True
    if (date.hour >= 10) and (date.minute == 5) and is_business_day(date):
        return True
        
    return False   

def flag_work_15min(date):
    name_15min = f"{str(pd.Timestamp.now()).split(':')[0].replace(' ', '_')}_{(pd.Timestamp.now().minute // 15) * 15}"  #date+hour+15min
    cur_round_15min = (pd.Timestamp.now().minute // 15) * 15
    if (date.hour >= 10) and (date.minute >= cur_round_15min) and is_business_day(date) and (not os.path.exists(f'./data/raw/15min/{name_15min}')):
        return True
    return False   



In [68]:
while 1 > 0:
    clear_output()
    cur_time = pd.Timestamp.now().floor('s')
    print(cur_time)
    
    if flag_work_1day(cur_time):
        get_data_1day(STOCKS)
        preproc_data_1day(STOCKS)
        
    if flag_work_1hour(cur_time):   
        get_data_1hour(STOCKS)
        preproc_data_1hour(STOCKS)

    if flag_work_15min(cur_time):
        get_data_15min(STOCKS)
        preproc_data_15min(STOCKS)    
        
        prepare_data_lgbm(STOCKS, FEATURES)
        predict_lgbm(MODELS)
        send_notification_tg(STOCKS)
        
    time.sleep(10)

2025-02-26 15:30:01


 81%|███████████████████████████████████████████████████████████              | 55/68 [00:24<00:05,  2.25it/s]


KeyboardInterrupt: 

In [ ]:
data = load_pkl('./data/result/data_result_2025-02-19_21_30.pkl')
data

In [ ]:
data['y_pred'].max()

In [ ]:
Часовая и 15 минутная дата в 10 часов (Error_time_data)

посмотреть, что там за дата

Может он и не должен отрабатывать в это время? хотя скорее там со временм не понятно